# 1、连接到spark

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .master("local[4]") \
    .getOrCreate()

# 2、创建dataframe

方式一：从文件系统中创建

有多种创建dataframe的方式，从文件系统中、hive的数据库表中、以及其他的spark data resource

In [3]:
df = spark.read.json("./resources/people.json")
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [7]:
print(type(df))

<class 'pyspark.sql.dataframe.DataFrame'>


三、Untyped Dataset Operations

In [ ]:
# 以树的形式打印schema
df.printSchema()
# 选择其中的name列
df.select("name").show()
# 选择所有列，对所有人的年龄+1
df.select(df["name"],df["age"]+1).show()
# 统计不同年龄的人数
df.groupBy("age").count().show()

四、Running SQL Queries Programmatically

In [ ]:
# Register the DataFrame as a SQL temporary view
# 将df注册为一个临时表，这个临时表是会话级别的，会话关闭后表会消失
df.createOrReplaceTempView("people")
# 使用sql语句
sqlDF = spark.sql("select * from people")
print(type(sqlDF))
sqlDF.show()

In [ ]:
# 下面的语句新创建了一个session，会报错
spark.newSession().sql("select * from people").show()

In [6]:
# 也可以创建一个全局的临时表，当spark关闭后消失，全局临时表存储在global_temp数据库中，引用其中的表时，必须包含namespace
df.createGlobalTempView("people")
spark.sql("select * from global_temp.people").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



# 五、Interoperating with RDDs（转换rdd到dataframe）

## 方式一：Inferring the Schema Using Reflection

In [6]:

# 1. 创建rdd
# 2. 将rdd中的数据通过map方法进行处理，以键值对的方式传输给row对象，其中key决定了dataframe的列名，value的type是根据数据集推断而来
# 3. 将2得到的结果传给createDataFrame，即可创建dataframe
## 下面是一个案例
from pyspark.sql import Row
# 通过sparksession，获取spark上下文
sc = spark.sparkContext
# 从本地文件创建rdd
rdd = sc.textFile("resources/people.txt")
print("数据类型:",type(rdd))
# 使用map操作生成row
# 切分数据
lines = rdd.map(lambda line:line.split(","))
people = lines.map(lambda line:Row(name=line[0],age=int(line[1])))
# 创建dataframe
df = spark.createDataFrame(people)
print("数据类型:",type(df))


数据类型: <class 'pyspark.rdd.RDD'>
数据类型: <class 'pyspark.sql.dataframe.DataFrame'>


In [10]:
df.printSchema()
# 创建session级别的临时表
df.createOrReplaceTempView("people")
teenagers = spark.sql("select * from people where age <= 19 and age >= 13")

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)



In [11]:
teenagers.show()

+------+---+
|  name|age|
+------+---+
|Justin| 19|
+------+---+



In [12]:
# 通过rdd的map方式，获取所有姓名
teenNames = teenagers.rdd.map(lambda p:"Name:"+p.name).collect()
print(teenNames)

['Name:Justin']


## 方式二，通过编程的方式显示的声明value的类型

In [16]:
# 1. 将原有的rdd转化为一个元组或者列表类型的rdd
# 2. 创建一个schema，这个schema的类型是StructureType；该实例由字段列表作为入参，每个字段由字段名、类型、是否允许为空作为入参
# 3. 创建dataframe

from pyspark.sql.types import StringType,StructField,StructType
# 1. 转化原有的rdd
# 通过sparkSession连接到上下文
sc = spark.sparkContext
# 从本地文件读取并创建rdd
rdd = sc.textFile("resources/people.txt")
# 将其转化为元组类型
# Justin, 19 -> (Justin,19)
lines = rdd.map(lambda line:line.split(','))
people = lines.map(lambda p:(p[0],p[1]))
# 定义schema,schema是一个structType对象，对象的参数是一个列表
# 列表中的每一个元素，均是一个 structField对象
# structField对象的参数，包括：字段名、字段类型，是否允许为空
# schemaString
schemaString = "name age"
# 创建一个字段列表
fileds = [StructField(filedName,StringType(),True)  for filedName in schemaString.split(" ")]
schemaPeople = StructType(fileds)

# 创建dataframe
people = spark.createDataFrame(people,schemaPeople)

In [17]:
people.createOrReplaceTempView("people")
spark.sql("select * from people").show()

+-------+---+
|   name|age|
+-------+---+
|Michael| 29|
|   Andy| 30|
| Justin| 19|
+-------+---+



In [21]:
people.select("name").count()

3

In [24]:
people.groupBy(["name","age"]).count().show()

+-------+---+-----+
|   name|age|count|
+-------+---+-----+
|   Andy| 30|    1|
|Michael| 29|    1|
| Justin| 19|    1|
+-------+---+-----+



In [22]:
people.select("name").distinct().show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+

